In [1]:
import os, shutil, zipfile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.font_manager as fm
font_name = fm.FontProperties(fname="C:\\Windows\\Fonts\\malgun.ttf").get_name()
plt.rc("font", family = font_name)

import matplotlib as mlp
mlp.rcParams["axes.unicode_minus"] = False

import networkx as nx
from networkx.drawing.nx_pydot import graphviz_layout
from apyori import apriori

import random
import re
import json

from urllib.request import urlopen
from bs4 import BeautifulSoup

# 에너지적정비율

- 서울시민건강포털
    - https://health.seoulmc.or.kr/healthCareInfo/nutrientStandard.do

## 크롤링 및 전처리

- 결측값을 0으로 대체
- 남자여자만 

In [2]:
##### 에너지 적정비율 저장
page = urlopen("https://health.seoulmc.or.kr/healthCareInfo/nutrientStandard.do")
soup = BeautifulSoup(page, "html.parser")

trs=soup.find_all("tbody")[0].find_all("tr")
cnt=0
text=[]
group=[]
for tr in range(len(trs)):
    tds = trs[tr].get_text().split('\n')[1:-1]    
    if len(tds)==7 :
        tds = tds
        group.append(tds[0])
    else :
        if tds[0] in ["임산부","수유부"] :
            group.append(tds[0])
            tds.insert(1, tds[0])                
        else :
            tds.insert(0,group[cnt-1])
            group.append(tds[0])
    text.append(tds)
    cnt=cnt+1
# print(text)

text_modify=[]
for td in text :
    tds =[]
    for i in range(len(td)) :
        if td[i] in ['-','\xa0'] :
            td[i]=0
        tds.append(td[i])
    text_modify.append(tds)   
# print(text_modify)

df_crawly=pd.DataFrame(text_modify, 
                    columns=["성별","연령별","탄수화물","단백질","지방",
                             "포화지방산","트랜스지방"])

df_crawly.to_csv('files/에너지적정비율_crawling.csv', header=True, 
              encoding="utf-8", index=False)

df_crawly.head()

,성별,연령별,탄수화물,단백질,지방,포화지방산,트랜스지방
0,영아,0-5(개월),0,0,0,0,0
1,영아,6-11,0,0,0,0,0
2,유아,1-2(세),55-65,7-20,0,20-35,1 미만
3,유아,3-5,55-65,7-20,15-30,8 미만,1 미만
4,남자,6-8(세),55-65,7-20,15-30,8 미만,1 미만


In [3]:
df_crawly=pd.read_csv('files/에너지적정비율_crawling.csv', 
                      encoding="utf-8")

In [4]:
##### 파일 저장

df_ratio = pd.DataFrame(columns=["성별","연령별","영양소",
                                 "하한(%)","상한(%)"])

val = ['탄수화물','단백질','지방']
cnt=0
ages=[]
for v in val :
    for i in df_crawly.index :
        if "(세)" in df_crawly["연령별"][i] :
            age=df_crawly["연령별"][i].replace("(세)","")
        elif df_crawly["연령별"][i] == '6-11' :
            age = df_crawly["연령별"][i]+'(개월)'
        else :
            age= df_crawly["연령별"][i]
        sex = df_crawly["성별"][i]
        cat=v+'(g)'

        if df_crawly[v][i]==0 :
            min_num=0
            max_num=0
        else :
            min_num=re.findall("^([0-9]*)", df_crawly[v][i])[0]
            max_num=re.findall("([0-9]*)$", df_crawly[v][i])[0]

        df_ratio.loc[cnt]=[sex,age,cat,min_num, max_num]
#         print(df_ratio.loc[cnt])
        cnt=cnt+1
                
df_ratio=df_ratio[df_ratio["성별"].isin(["남자", "여자"])].reset_index(drop=True)

df_ratio.to_csv('files/에너지적정비율_탄단지(ratio).csv', 
                header=True, encoding="utf-8", index=False)

df_ratio.head()

,성별,연령별,영양소,하한(%),상한(%)
0,남자,6-8,탄수화물(g),55,65
1,남자,9-11,탄수화물(g),55,65
2,남자,12-14,탄수화물(g),55,65
3,남자,15-18,탄수화물(g),55,65
4,남자,19-29,탄수화물(g),55,65


---

# 일일섭취권장량

- 영양소 섭취기준을 통해 보는 한국인 영양소 섭취 현황(보도자료)
    - 2020 한국인 영양소 섭취기준 요약표 참고
    - 결측값을 0으로 대체
    -남자여자만

In [5]:
df_raw = pd.read_excel("files/raw_일일섭취권장량_탄단지.xlsx",
                         sheet_name='Sheet1')
##### 결측치 채우기
df_raw = df_raw.fillna(0)

##### 성별 통일
df_raw=df_raw[df_raw["성별"].isin(["남자", "여자"])].reset_index(drop=True)

##### 연령별 통일
for i in df_raw.index :
    if "(세)" in df_raw["연령"][i] :
        df_raw["연령"][i]=df_raw["연령"][i].replace("(세)","")
    else :
        df_raw["연령"][i]=df_raw["연령"][i]
df_raw=df_raw.rename(columns={"연령" : "연령별"})

##### 영양소 통일
df_raw["영양소"]=df_raw["영양소"].str.replace("/일","")
df_raw.head()

##### 에너지와 영양소 분리
df_energe=df_raw[df_raw["영양소"]=="에너지(kcal)"].reset_index(drop=True)
df_energe.drop(['영양소','권장섭취량','충분섭취량','상한섭취량'], axis=1, inplace=True)
df_energe=df_energe.rename(columns={"평균필요량":"에너지(kcal)"})
df_energe.head()

df_amount=df_raw[df_raw["영양소"]!="에너지(kcal)"].reset_index(drop=True)
df_amount.drop(['평균필요량','충분섭취량','상한섭취량'], axis=1, inplace=True)
df_amount.tail()

##### 영양소df에 총당류(에너지의 10%/4kcal), 나트륨(2000mg=2g) 추가
cnt=len(df_amount)
# print(cnt)
for c in ["총당류(g)", "나트륨(g)"] :
    for i in df_energe.index:
        if c == "총당류(g)" :
#             print(df_kcal["평균필요량"][i])
            new = df_energe["에너지(kcal)"][i]/40
        else :
            new = 2
            
#         print([df_kcal["성별"][i],df_kcal["연령별"][i],c,0,new,0,0])
            
        row=[df_energe["성별"][i],df_energe["연령별"][i],c,new]
#         print(cnt, row)
        df_amount.loc[cnt]=row
        cnt=cnt+1  
df_amount.tail()

##### 에너지 영양소 머지
df_nutrient = pd.merge(df_energe, df_amount, on=["성별","연령별"], how='outer')
df_nutrient.fillna(0)

df_nutrient.to_csv("files/일일영양소권장섭취량(nutrient).csv",
                encoding="utf-8", index=False)

df_nutrient.head()

C:\Users\acorn\AppData\Local\Temp\ipykernel_13536\610851292.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw["연령"][i]=df_raw["연령"][i].replace("(세)","")
C:\Users\acorn\AppData\Local\Temp\ipykernel_13536\610851292.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw["연령"][i]=df_raw["연령"][i]


,성별,연령별,에너지(kcal),영양소,권장섭취량
0,남자,6-8,1700.0,탄수화물(g),130.0
1,남자,6-8,1700.0,지방(g),0.0
2,남자,6-8,1700.0,단백질(g),35.0
3,남자,6-8,1700.0,총당류(g),42.5
4,남자,6-8,1700.0,나트륨(g),2.0


# 지방 섭취량 구하기

- 지방 권장 섭취량이 0으로 나와 에너지에서 지방의 비율의 최대치를 대체
- 상하한비율: 에너지*영양소비율
- 상하한비율/1g당 칼로리(지방 9, 나머지 4)
- 당류와 나트륨의 상한은 권장량 그대로

In [6]:
df_minmax=pd.merge(df_nutrient, df_ratio, on=["성별","연령별","영양소"], how='outer')
df_minmax=df_minmax.fillna(0)
df_minmax.head()

df_minmax['하한권장량']=''
df_minmax['상한권장량']=''

##### 상한하한 구하기

for i in df_minmax.index :
    if df_minmax["영양소"][i] == "나트륨(g)" :
        max_num=2
        min_num=0
        
    elif df_minmax["영양소"][i] == "총당류(g)" :
        dang_term=df_minmax[df_minmax["영양소"]=="총당류(g)"]
        max_num=dang_term["권장섭취량"][i]
        min_num=0
        
    else :
        if df_minmax["영양소"][i] == "지방(g)" :
            kcalby=9
        else :
            kcalby=4
        
        energe=df_minmax["에너지(kcal)"][i]
        max_r=float(df_minmax["상한(%)"][i])
        min_r=float(df_minmax["하한(%)"][i])
#         print(energe, max_r, min_r)
#         print(type(energe), type(max_r), type(min_r))
        max_num=(energe*(max_r/100))/kcalby
        min_num=(energe*(min_r/100))/kcalby
        
#     print(max_num, min_num)
        
    df_minmax["상한권장량"][i]=max_num
    df_minmax["하한권장량"][i]=min_num

df_minmax[df_minmax["영양소"]=="총당류(g)"]

df_minmax.to_csv('files/일일권장섭취량(minmax).csv',
                   encoding='utf-8', index=False)

df_minmax.head()

##### 지방상한을 대체하기
df_intake=df_minmax.copy()
df_intake

fat_term = df_intake[df_intake["영양소"]=="지방(g)"]
for i in fat_term.index :
    df_intake['권장섭취량'][i] = df_intake['상한권장량'][i]


df_intake[df_intake["영양소"]=="지방(g)"]

df_intake.drop(["하한(%)","상한(%)","하한권장량","상한권장량"], axis=1, inplace=True)

df_intake.to_csv("content/권장섭취량(intake).csv",
                encoding="utf-8", index=False)

df_intake.head()

C:\Users\acorn\AppData\Local\Temp\ipykernel_13536\2833755488.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_minmax["상한권장량"][i]=max_num
C:\Users\acorn\AppData\Local\Temp\ipykernel_13536\2833755488.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_minmax["하한권장량"][i]=min_num
C:\Users\acorn\AppData\Local\Temp\ipykernel_13536\2833755488.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_intake['권장섭취량'][i] = df_intake['상한

,성별,연령별,에너지(kcal),영양소,권장섭취량
0,남자,6-8,1700.0,탄수화물(g),130.000000
1,남자,6-8,1700.0,지방(g),56.666667
2,남자,6-8,1700.0,단백질(g),35.000000
3,남자,6-8,1700.0,총당류(g),42.500000
4,남자,6-8,1700.0,나트륨(g),2.000000


---

# 식품영양성분DB

- https://various.foodsafetykorea.go.kr/nutrient/

In [7]:
df_element = pd.read_excel('content/Nutri_food_DB.xlsx',sheet_name="20230215")

df_element = df_element.sort_values('연도', ascending=False) # 연도별로 내림차순 정렬
df_element = df_element.drop_duplicates('식품명') # 식품명 중복 제거

df_food=df_element.loc[df_element['지역 / 제조사']=="전국(대표)", 
           ['식품명','식품대분류','1회제공량',
            '에너지(㎉)', '탄수화물(g)' ,'단백질(g)','지방(g)',
           '총당류(g)','나트륨(㎎)']]

df_food=df_food.rename(columns={'에너지(㎉)':'에너지(kcal)',
                               '나트륨(㎎)':'나트륨(g)'})

##### 나트륨 단위 변경
for i in df_food.index:
    df_food['나트륨(g)'][i] = df_food['나트륨(g)'][i]/1000

# print(df_food["식품명"].value_counts()) # 멸치풋고추볶음,우동(일식) 중복
# print('====================================================')
# print(df_food[(df_food["식품명"].isin(['멸치풋고추볶음','우동(일식)'])) & 
#              (df_food["연도"]==2019)].index)

# df_food.drop([87, 154], inplace=True)
# del df_food["연도"]

##### 메인메뉴 여부 매칭
df_main_menu = pd.read_table("content/final_main_menu.txt", 
                             encoding="utf-8", header=None)

# print(len(df_main_menu))
mains=df_main_menu.values.tolist()
# print(mains)
main_menu=[i[0] for i in mains]
# print(main_menu)
# print(len(main_menu))

food_mains=[]
for i in df_food["식품명"]:
#     print(i)
    if i in main_menu :
        food_mains.append("main")
    else : 
        food_mains.append("not main")
        
df_food["메인여부"]=food_mains
        
df_food.to_csv("content/식품영양정보_탄단지_main(food).csv", 
               encoding="utf-8", index=False)
        
print(len(df_food[df_food["메인여부"]=="main"]))

df_food.head()

218


C:\Users\acorn\AppData\Local\Temp\ipykernel_13536\368485709.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_food['나트륨(g)'][i] = df_food['나트륨(g)'][i]/1000


,식품명,식품대분류,1회제공량,에너지(kcal),탄수화물(g),단백질(g),지방(g),총당류(g),나트륨(g),메인여부
7063,잔멸치고추볶음,볶음류,50,114.05,2.86,16.64,4,1.88,1.03052,not main
7074,멸치풋고추볶음,볶음류,50,94.83,6.65,10.31,3,4.51,0.67244,not main
7073,팔보채,볶음류,220,196.98,7.53,27.33,6.39,4.59,1.88399,main
7072,콩나물잡채,볶음류,230,153.12,28.86,4.43,2.22,21.56,1.42294,not main
7071,채소떡볶이,볶음류,200,515.29,114.11,6.24,3.76,3.94,0.53797,main


In [8]:
# 전처리된 데이터프레임 불러오기

df_intake=pd.read_csv("content/권장섭취량(intake).csv",encoding="utf-8")
df_food=pd.read_csv("content/식품영양정보_탄단지_main(food).csv", encoding="utf-8")